### Parse `Husky`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import r2rpy
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


/home/rilab/.pyenv/versions/3.8.13/envs/llm/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Parse `scene_husky_w_ur5_rg2_d435i.xml`

In [2]:
xml_path = '../../asset/husky/scene_husky_w_ur5_rg2_d435i.xml'
env = MuJoCoParserClass(name='Husky',rel_xml_path=xml_path,VERBOSE=True)
print ("[%s] parsed."%(env.name))

dt:[0.0020] HZ:[500]
n_dof (=nv):[22]
n_geom:[59]
geom_names:['floor', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
n_body:[24]
body_names:['world', 'base_husky', 'front_left_wheel_link', 'front_right_wheel_link', 'rear_left_wheel_link', 'rear_right_wheel_link', 'base_ur5e', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'tcp_link', 'camera_mount', 'd435i', 'rg2_gripper_base_link', 'camera_center', 'rg2_gripper_finger1_finger_link', 'rg2_gripper_finger1_inner_knuckle_link', 'rg2_gripper_finger1_finger_tip_link', 'rg2_gripper_finger2_finger_link', 'rg2_gripper_finger2_inner_knuckle_link', 'rg2_gripper_finger2_finger_tip_link']
n_joint:[17

### Render

In [4]:
env.init_viewer(viewer_title='Husky',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=107.08,distance=2.5,elevation=-27,lookat=[0.4,0.3,0.2],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.1,contactheight=0.1,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
# Initial UR5e Pose
init_ur_q = np.array([np.deg2rad(-90), np.deg2rad(-132.46), np.deg2rad(122.85), np.deg2rad(99.65), np.deg2rad(45), np.deg2rad(-90.02)])

# Change controller gain
for ctrl_name in env.ctrl_names:
    kv = 1
    # env.model.actuator(ctrl_name).gainprm[0] = kv
    # env.model.actuator(ctrl_name).biasprm[2] = -kv
env.reset()
while (env.get_sim_time() < 100.0) and env.is_viewer_alive():
    # Step
    ctrl_husky = 5*np.array([-1,-1,-1,-1]) # rotate:[5,-5,5,-5] straight:[5,5,5,5]
    ctrl_wholebody = ctrl_husky.tolist() + init_ur_q.tolist()
    # env.step(ctrl=ctrl_husky, ctrl_idxs=[0,1,2,3])
    env.step(ctrl=ctrl_wholebody, ctrl_idxs=[0,1,2,3,4,5,6,7,8,9])
    # Do render
    if env.loop_every(HZ=30) or (env.tick == 1):
        # Update viewer information
        p_base = env.get_p_body(body_name='base_husky')
        env.update_viewer(lookat=p_base,CALL_MUJOCO_FUNC=False)
        # Render contact information
        PLOT_CONTACT = True
        if PLOT_CONTACT:
            p_contacts,f_contacts,geom1s,geom2s, _,_ = env.get_contact_info()
            for (p_contact,f_contact,geom1,geom2) in zip(p_contacts,f_contacts,geom1s,geom2s):
                f_norm = np.linalg.norm(f_contact)
                f_uv = f_contact / (f_norm+1e-8)
                f_len = 0.3 # f_norm*0.05
                label = '' #'[%s]-[%s]'%(geom1,geom2)
                # env.plot_arrow(p=p_contact,uv=f_uv,r_stem=0.01,len_arrow=f_len,rgba=[1,0,0,0.4],label='')
                # env.plot_arrow(p=p_contact,uv=-f_uv,r_stem=0.01,len_arrow=f_len,rgba=[1,0,0,0.4],label='')
                # env.plot_sphere(p=p_contact,r=0.0001,label=label)
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),PLOT_AXIS=True,axis_len=1.0,axis_width=0.01)
        env.plot_body_T(body_name='base_husky',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='front_left_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='front_right_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='rear_left_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='rear_right_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_T(p=p_base+np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label='[%.2f]sec'%(env.get_sim_time()))
        env.render()
        
# Close viewer
env.close_viewer()
print ("Time:[%.2f]sec. Done."%(env.get_sim_time()))

Pressed ESC
Quitting.
Time:[4.55]sec. Done.


### Open interactive viewer

In [7]:
env.open_interactive_viewer()